In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import pandas as pd
from PIL import Image
import os
from tqdm import tqdm

In [7]:
class ChestXrayDataset(Dataset):
    def __init__(self, image_folder, csv_file, transform=None):
        self.image_folder = image_folder
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_folder, f'img_{self.data.iloc[idx, 0]}.png')
        image = Image.open(img_name).convert('L')
        if self.transform:
            image = self.transform(image)
        label = int(self.data.iloc[idx, 1])
        return image, label

In [8]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = None
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        return out


class ResNet16(nn.Module):
    def __init__(self):
        super(ResNet16, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)  
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(64, 64, blocks=4, stride=1)
        self.layer2 = self._make_layer(64, 128, blocks=4, stride=2)
        self.layer3 = self._make_layer(128, 256, blocks=4, stride=2)
        self.layer4 = self._make_layer(256, 512, blocks=4, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, 3)  

    def _make_layer(self, in_channels, out_channels, blocks, stride):
        layers = []
        layers.append(ResidualBlock(in_channels, out_channels, stride))
        for _ in range(1, blocks):
            layers.append(ResidualBlock(out_channels, out_channels, stride=1))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


In [9]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_images_folder = "data/train_images/"
train_csv_file = "data/train_answers.csv"
batch_size = 32
num_epochs = 20

train_dataset = ChestXrayDataset(train_images_folder, train_csv_file, transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

model = ResNet16().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [10]:
model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    tqdm_loader = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}')
    for images, labels in tqdm_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
        tqdm_loader.set_postfix({'Loss': loss.item()})
    epoch_loss = running_loss / len(train_dataset)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}')

Epoch 1/20: 100%|██████████| 844/844 [1:27:58<00:00,  6.25s/it, Loss=0.547]


Epoch [1/20], Loss: 0.7392


Epoch 2/20: 100%|██████████| 844/844 [1:33:38<00:00,  6.66s/it, Loss=0.236]


Epoch [2/20], Loss: 0.4966


Epoch 3/20: 100%|██████████| 844/844 [1:32:22<00:00,  6.57s/it, Loss=0.344] 


...

Epoch [18/20], Loss: 0.1403


Epoch 19/20: 100%|██████████| 844/844 [1:08:08<00:00,  4.84s/it, Loss=0.16]  


Epoch [19/20], Loss: 0.1300


Epoch 20/20: 100%|██████████| 844/844 [1:13:39<00:00,  5.24s/it, Loss=0.225] 

Epoch [20/20], Loss: 0.1206


In [14]:
torch.save(model.state_dict(), 'resnet16_20.pth')

In [13]:
submission_df = pd.DataFrame(columns=['id', 'target_feature'])
test_images_folder = "data/test_images/"
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

submission_data = []

for i, filename in enumerate(os.listdir(test_images_folder)):
    img_path = os.path.join(test_images_folder, filename)
    img = Image.open(img_path).convert('L')
    img = test_transform(img).unsqueeze(0).to(device)
    output = model(img)
    _, predicted = torch.max(output, 1)
    if predicted.item() is None or predicted.item() == '':
        target_feature = 0
    else:
        target_feature = predicted.item()
    submission_data.append({'id': i, 'target_feature': target_feature})

submission_df = pd.DataFrame(submission_data)
submission_df.to_csv('submission_file-resnet16_20.csv', index=False)

In [15]:
model.train()
for epoch in range(10):
    running_loss = 0.0
    tqdm_loader = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}')
    for images, labels in tqdm_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
        tqdm_loader.set_postfix({'Loss': loss.item()})
    epoch_loss = running_loss / len(train_dataset)
    torch.save(model.state_dict(), f'resnet16_20_{epoch + 20}.pth')
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}')

Epoch 1/20: 100%|██████████| 844/844 [1:13:30<00:00,  5.23s/it, Loss=0.117]  


Epoch [1/20], Loss: 0.1103


Epoch 2/20: 100%|██████████| 844/844 [1:13:16<00:00,  5.21s/it, Loss=0.0442]


Epoch [2/20], Loss: 0.1001


Epoch 3/20: 100%|██████████| 844/844 [1:13:16<00:00,  5.21s/it, Loss=0.0187] 


...

Epoch 7/20: 100%|██████████| 844/844 [1:13:10<00:00,  5.20s/it, Loss=0.00324]


Epoch [7/20], Loss: 0.0593


Epoch 8/20: 100%|██████████| 844/844 [1:13:29<00:00,  5.22s/it, Loss=0.135]   


Epoch [8/20], Loss: 0.0526


Epoch 9/20:   7%|▋         | 57/844 [06:00<1:23:02,  6.33s/it, Loss=0.0031] 

KeyboardInterrupt



In [16]:
submission_df = pd.DataFrame(columns=['id', 'target_feature'])
test_images_folder = "data/test_images/"
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

submission_data = []

for i, filename in enumerate(os.listdir(test_images_folder)):
    img_path = os.path.join(test_images_folder, filename)
    img = Image.open(img_path).convert('L')
    img = test_transform(img).unsqueeze(0).to(device)
    output = model(img)
    _, predicted = torch.max(output, 1)
    if predicted.item() is None or predicted.item() == '':
        target_feature = 0
    else:
        target_feature = predicted.item()
    submission_data.append({'id': i, 'target_feature': target_feature})

submission_df = pd.DataFrame(submission_data)
submission_df.to_csv('submission_file-resnet16_30.csv', index=False)